In [1]:
corpus= [
    "hello there a good man!",
    "it is quite windy in London",
    "how is good the weather today?"
]

In [8]:
tokenized_corpus=[]
for doc in corpus:
    print(doc)
    print (doc.split(" "))

hello there a good man!
['hello', 'there', 'a', 'good', 'man!']
it is quite windy in London
['it', 'is', 'quite', 'windy', 'in', 'London']
how is good the weather today?
['how', 'is', 'good', 'the', 'weather', 'today?']


In [15]:
doc1=corpus[0].split()

In [16]:
tokenized_corpus.append(doc1)

In [17]:
tokenized_corpus

[['hello', 'there', 'a', 'good', 'man!']]

In [19]:
!pip install python-docx


  Using cached python_docx-1.1.2-py3-none-any.whl.metadata (2.0 kB)
Using cached python_docx-1.1.2-py3-none-any.whl (244 kB)


In [20]:
import os
import pandas as pd
import fitz  # pymupdf
from docx import Document

# Liste pour stocker les CV
resumes = []

# Dossier contenant les CV
folder_path = "cv_test"

# Parcourir les fichiers du dossier
for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
    text = ""  # Initialiser le texte pour chaque fichier

    if filename.endswith(".pdf"):
        doc = fitz.open(file_path)
        for page in doc:
            text += page.get_text("text")  # Extraire le texte de chaque page
    elif filename.endswith(".docx"):
        doc = Document(file_path)
        for para in doc.paragraphs:
            text += para.text + "\n"  # Ajouter chaque paragraphe

    # Ajouter le fichier et son contenu à la liste
    resumes.append({"filename": filename, "content": text})

# Convertir la liste de dictionnaires en DataFrame
df_resumes = pd.DataFrame(resumes)

# Afficher le DataFrame
print(df_resumes)


                                             filename  \
0                          CV-_Taoufik_GHERRI_PM_.pdf   
1           CV999735_Fonkouo-Bienvenu_Turnover-it.pdf   
2         CVarchitectecteCyberSécurité (1) (1).docx   
3             CVarchitectecteCyberSécurité (1).docx   
4                     CV_2025-02-Pierre_LEFEBVRE_.pdf   
5               CV_Ilies_Maarouf_V5.pdf.249887956.pdf   
6                       CV_JIRARI_MEHDI_FINAL (1).pdf   
7                               CV_MHATLIaziz (1).pdf   
8                               CV_Nihel_Ben_Hsan.pdf   
9         document-55-5566-architecte-it-0089e7da.pdf   
10                      document-5TdPlsM4Zy8j9agF.pdf   
11  document-adnane-khirchi-data-analyst-engineer-...   
12  document-alhassane-balde-ingenieur-devops-de-p...   
13                 document-cloud-devops-cbba25d8.pdf   
14  document-hamza-essid-business-analyst-chef-de-...   
15  document-imghi-abdelkarim-developpeur-java-kaf...   
16  document-nabil-ouassou-inge

In [22]:
import re
import nltk
import string
from nltk.corpus import stopwords

# Télécharger les stopwords si ce n'est pas déjà fait
nltk.download('stopwords')
# Charger les stopwords en français et en anglais
stopword = set(stopwords.words('french') + stopwords.words('english'))
def clean_data(text):
    # Remplacer plusieurs retours à la ligne par un espace
    text = re.sub(r'\n+', ' ', text).strip()
    # Transformer en minuscules
    text = text.lower()
    # Supprimer la ponctuation
    text = text.translate(str.maketrans("", "", string.punctuation))
    # Supprimer les stopwords
    text = " ".join(word for word in text.split() if word not in stopword)
    return text

[nltk_data] Downloading package stopwords to C:\Users\Nour
[nltk_data]     chkiwa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [23]:
df_resumes_cleaned = df_resumes.copy()

# Appliquer la fonction `clean_data` sur la copie
df_resumes_cleaned['content'] = df_resumes_cleaned['content'].apply(clean_data)

# Afficher les 5 premières lignes pour vérifier
print(df_resumes_cleaned.head())

                                      filename  \
0                   CV-_Taoufik_GHERRI_PM_.pdf   
1    CV999735_Fonkouo-Bienvenu_Turnover-it.pdf   
2  CVarchitectecteCyberSécurité (1) (1).docx   
3      CVarchitectecteCyberSécurité (1).docx   
4              CV_2025-02-Pierre_LEFEBVRE_.pdf   

                                             content  
0  taoufik gherri chef projet analyste fonctionne...  
1  tel 237675696243 mail fondev88gmailcom 1 bienv...  
2  dernier diplôme date 2018 isaca’s cybersecurit...  
3  dernier diplôme date 2018 isaca’s cybersecurit...  
4  pierre lefebvre chef projet senior ecommerce d...  


In [33]:
filenames = df_resumes_cleaned['filename'].to_list()
corpus = df_resumes_cleaned['content'].to_list()

In [34]:
len(corpus)

23

In [35]:
tokenized_corpus=[]
for doc in corpus:
    tokenized_doc = doc.split()
    tokenized_corpus.append(tokenized_doc)

In [28]:
!pip install rank_bm25

In [40]:
from rank_bm25 import BM25Okapi

bm25 = BM25Okapi(tokenized_corpus)

In [52]:
query="Openshift"
query=clean_data(query)
top_docs=bm25.get_top_n(query, corpus, n=5)
top_filenames = [filenames[corpus.index(doc)] for doc in top_docs]
print(top_filenames)


['document-5TdPlsM4Zy8j9agF.pdf', 'document-hamza-essid-business-analyst-chef-de-projet-moa-c4077aff.pdf', 'CV_Ilies_Maarouf_V5.pdf.249887956.pdf', 'CV_MHATLIaziz (1).pdf', 'CV_2025-02-Pierre_LEFEBVRE_.pdf']


In [53]:
query

'openshift'

In [69]:
query = ["Openshift", "Kubernetes", "System Linux", "Automatisation"]
query = [clean_data(q) for q in query] 

top_docs=bm25.get_top_n(query, corpus, n=5)
top_filenames = [filenames[corpus.index(doc)] for doc in top_docs]
print(top_filenames)


['Ingénieur  OpenShift, Kubernetes, Linux (2).docx', 'Nesrine-MalahCV (2).pdf', 'CV_JIRARI_MEHDI_FINAL (1).pdf', 'CV_Ilies_Maarouf_V5.pdf.249887956.pdf', 'document-cloud-devops-cbba25d8.pdf']


In [63]:
query

['openshift', 'kubernetes', 'system linux', 'automatisation']

In [64]:
# Nettoyer et tokenizer chaque terme de la requête
query = ["Openshift", "Kubernetes", "System Linux", "Automatisation"]
query = [clean_data(q).split() for q in query]  # Nettoyage et tokenization
query = [word for sublist in query for word in sublist]  # Aplatir la liste

# Appliquer BM25
top_docs = bm25.get_top_n(query, corpus, n=5)

# Récupérer les noms de fichiers correspondants
top_filenames = [filenames[corpus.index(doc)] for doc in top_docs]

print(top_filenames)


['WFE [Architecte DevOps - Expert Système, RHCA®, Ansible®, Virtualisation® &amp_ Cloud, Openshift, Satellite ®, Containers] CV - RIDCHA DATA.docx', 'document-alhassane-balde-ingenieur-devops-de-production-0a992c89.pdf', 'Ingénieur  OpenShift, Kubernetes, Linux (2).docx', 'CV_Ilies_Maarouf_V5.pdf.249887956.pdf', 'Nesrine-MalahCV (2).pdf']


In [65]:
query = ["Openshift", "Kubernetes", "System Linux", "Automatisation"]
query = [clean_data(q).split() for q in query]
query

[['openshift'], ['kubernetes'], ['system', 'linux'], ['automatisation']]

In [66]:
query='''  Objectif global :Administrer les clusters Openshift du client


Compétences techniques :
- Openshift - Expert - Impératif
- Kubernetes - Confirmé - Impératif
- System Linux - Confirmé - Impératif
- Automatisation - Confirmé - Impératif


Connaissances linguistiques :
- Anglais Professionnel (Impératif)


Description détaillée :
- Administration d'une infrastructure Openshift en expansion.
- MCO des plates-formes d'hébergement Openshift.
- Analyse et résolution d'incident niveau 2 et 3.
- Conseil et expertise.
- Astreintes et interventions en HNO.




Définition du profil :


•⁠ Défini et gère l'évolution de l'architecture du système informatique en alignement avec la stratégie de l'entreprise.
•⁠  ⁠Dans le cadre des projets, défini l'architecture de la solution conformément aux exigences recueillies et en cohérence avec le SI.
•⁠  ⁠En fonction du projet, peut être le garant de la définition globale de la solution technique et de sa mise en œuvre en collaboration avec l'ensemble des acteurs techniques du projet.
•⁠  ⁠Elabore des cibles d'évolution du système informatique en adéquation avec les orientations stratégiques de l'entreprise.
•⁠  ⁠Défini et maintien le référentiel d'architecture technique de l'entreprise.
•⁠  ⁠Assure la vision globale de la solution, coordonner les travaux d'architecture et les différents contributeurs techniques, dans le cadre des projets
•⁠  ⁠Conçoit/ construit l'architecture des projets conformément aux exigences, à la cohérence globale avec le système informatique et les orientations définies par l'entreprise


Type de prestation : AT


Durée ou période : Période


Début Prestation : 03/03/2025


Fin Prestation : 28/02/2027
Lieu : Charenton/Paris / 5 jours par semaine sur site”
 '''
query = clean_data(query) 

top_docs=bm25.get_top_n(query, corpus, n=5)
top_filenames = [filenames[corpus.index(doc)] for doc in top_docs]
print(top_filenames)

['document-5TdPlsM4Zy8j9agF.pdf', 'CV_Ilies_Maarouf_V5.pdf.249887956.pdf', 'document-hamza-essid-business-analyst-chef-de-projet-moa-c4077aff.pdf', 'CV_2025-02-Pierre_LEFEBVRE_.pdf', 'document-alhassane-balde-ingenieur-devops-de-production-0a992c89.pdf']
